In [35]:
!pip install scikit-learn==1.3.2  # Machine Learning e funções de pré-processamento de dados.

In [36]:
## Importe bibliotecas relevantes para processamento e visualização de dados

import numpy as np                # álgebra linear
import pandas as pd               # processamento de dados, I/O de arquivo de conjunto de dados (por exemplo, pd.read_csv)
import matplotlib.pyplot as plt   # visualização de dados e plotagem gráfica
import seaborn as sns             # para visualizar distribuições aleatórias
%matplotlib inline

##  Adicione bibliotecas adicionais para preparar e executar o modelo

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesRegressor

pd.options.display.float_format = '{:.4f}'.format

import warnings                    # para lidar com mensagens de aviso
warnings.filterwarnings('ignore')

In [37]:
## Tenha baixado préviamente o conjunto de dados e feito o upload para o google colab
df = pd.read_csv('machinetable.csv')
df.head()

,id,created_at,updated_at,is_active,price,brand,url,crawl_date,year_reference,month_reference,...,model,year_fabrication,year_model,title,mileage,gear,fuel,bodywork,city,state
0,69c52d79-667f-4d7b-800e-e20f06ebc3c2,2025-01-18 23:14:24.230645-03:00,2025-01-18 23:14:24.230651-03:00,True,7699000.0000,HONDA,https://www.usadosbr.com//carros-e-utilitarios...,2025-01-18 23:14:24.230660-03:00,2025,1,...,CITY,NaN,2017,HONDA CITY,80000.0000,CVT,FLEX,SEDAN,CUIABA,MT
1,f0f2b352-757c-40e6-bcaf-de1192383d9c,2025-01-18 23:14:27.002594-03:00,2025-01-18 23:14:27.002601-03:00,True,6999000.0000,CHEVROLET,https://www.usadosbr.com//carros-e-utilitarios...,2025-01-18 23:14:27.002613-03:00,2025,1,...,ONIX,NaN,2023,CHEVROLET ONIX,69902.0000,MANUAL,FLEX,HATCH,GOIANIA,GO
2,8e97ce37-b3e6-4db1-8a65-d603ae1ac62c,2025-01-18 23:16:30.360775-03:00,2025-01-18 23:16:30.360782-03:00,True,6999000.0000,HYUNDAI,https://www.usadosbr.com//carros-e-utilitarios...,2025-01-18 23:16:30.360794-03:00,2025,1,...,HB20,NaN,2022,HYUNDAI HB20,44744.0000,MANUAL,FLEX,HATCH,SAO PAULO,SP
3,a4c59228-626e-406f-9036-d34360f8ffd5,2025-01-18 23:14:22.416528-03:00,2025-01-18 23:14:22.416546-03:00,True,5250000.0000,FORD,https://www.usadosbr.com//carros-e-utilitarios...,2025-01-18 23:14:22.416561-03:00,2025,1,...,ECOSPORT,NaN,2013,FORD ECOSPORT,228000.0000,MANUAL,FLEX,SUV,IBIUNA,SP
4,185d7281-7322-4d28-a11e-b962d692d532,2025-01-18 23:14:22.416587-03:00,2025-01-18 23:14:22.416593-03:00,True,17990000.0000,CHEVROLET,https://www.usadosbr.com//carros-e-utilitarios...,2025-01-18 23:14:22.416602-03:00,2025,1,...,S10,NaN,2017,CHEVROLET S10,141000.0000,AUTOMATICO,DIESEL,PICAPE CABINE DUPLA,GOIANIA,GO


In [38]:
# Contar valores ausentes por coluna
missing_values = df.isna().sum()

# Mostrar valores ausentes para cada coluna
print(missing_values)

id                     0
created_at             0
updated_at             0
is_active              0
price                  0
brand                  0
url                    0
crawl_date             0
year_reference         0
month_reference        0
description         4846
code_model             0
model                  0
year_fabrication    4846
year_model             0
title                  0
mileage                5
gear                   0
fuel                   0
bodywork               0
city                   0
state                  0
dtype: int64


In [39]:
# Descartar registros onde a coluna 'mileage' está ausente
df = df.dropna(subset=['mileage'])

# Confirmar a remoção
print(f"Número Total de Registros após descartar ausentes em 'mileage': {df.shape[0]}")

Número Total de Registros após descartar ausentes em 'mileage': 4841


In [40]:
## descartando as features irrelevantes (colunas)
df.drop(["id", "created_at", "updated_at", "is_active", "crawl_date", "url",
         "year_reference", "month_reference", "code_model", "year_fabrication",
         "description", "title"], axis=1, inplace=True)

## verificando as features após descartar recursos indesejados
df.columns

Index(['price', 'brand', 'model', 'year_model', 'mileage', 'gear', 'fuel',
       'bodywork', 'city', 'state'],
      dtype='object')

In [41]:
import datetime

# Idade do carro
current_year = datetime.datetime.now().year
df['age_years'] = current_year - df['year_model']

# diferença entre o preço de um veículo e o preço médio dos veículos com o mesmo modelo e ano modelo
df['price_deviation'] = df['price'] - df.groupby(['model', 'year_model'])['price'].transform('mean')

# Preço médio por marca
brand_avg = df.groupby('brand')['price'].transform('mean')
df['brand_avg_price'] = brand_avg

# Preço médio por estado
state_avg = df.groupby('state')['price'].transform('mean')
df['state_avg_price'] = state_avg

# Preço médio por cidade
city_avg = df.groupby('city')['price'].transform('mean')
df['city_avg_price'] = city_avg

# Marcas de luxo (ajuste conforme necessário)
luxury_brands = ['PORSCHE', 'JAGUAR', 'MERCEDES-BENZ', 'BMW', 'LAND ROVER', 'VOLVO']
df['is_luxury_brand'] = df['brand'].apply(lambda x: 1 if x in luxury_brands else 0)

In [42]:
# Cálculo do IQR para detectar outliers
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1

# Limites inferior e superior
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

# Filtrando os outliers
df_clean = df[(df['price'] >= lower_limit) & (df['price'] <= upper_limit)]

# Verificando os resultados
print(f"Dados originais: {len(df)} registros")
print(f"Dados após remoção dos outliers: {len(df_clean)} registros")

# Exibindo as estatísticas descritivas do dataframe sem outliers
print(df_clean['price'].describe())

Dados originais: 4841 registros
Dados após remoção dos outliers: 4670 registros
count       4670.0000
mean     9834804.9550
std      2884612.4249
min      1050000.0000
25%      7399000.0000
50%      9229000.0000
75%     11999000.0000
max     19355000.0000
Name: price, dtype: float64


In [43]:
df.describe()

,price,year_model,mileage,age_years,price_deviation,brand_avg_price,state_avg_price,city_avg_price,is_luxury_brand
count,4841.0000,4841.0000,4841.0000,4841.0000,4841.0000,4841.0000,4841.0000,4841.0000,4841.0000
mean,10699729.3782,2022.0409,47920.1161,2.9591,-0.0000,10699729.3782,10699729.3782,10699729.3782,0.0235
std,6753808.5676,2.4168,25419.0706,2.4168,974919.7594,4551482.7749,2261600.8541,2831950.3753,0.1517
min,1050000.0000,1973.0000,0.0000,0.0000,-15545000.0000,4000000.0000,6496750.0000,2059900.0000,0.0000
25%,7399000.0000,2022.0000,40382.0000,2.0000,-507001.3208,8808720.0663,9799506.0524,9694394.3916,0.0000
50%,9449000.0000,2023.0000,46278.0000,2.0000,-25075.9494,9845740.7286,9799506.0524,9939429.8555,0.0000
75%,12189000.0000,2023.0000,51443.0000,3.0000,261920.0000,10100532.5645,10632847.1545,10881728.7671,0.0000
max,175000000.0000,2025.0000,315913.0000,52.0000,15545000.0000,63618804.7619,28000000.0000,69995000.0000,1.0000


In [44]:
import pickle

In [45]:
# Supondo que 'df' já tenha as novas features criadas.

# 1. Identificar automaticamente as colunas categóricas e numéricas
categorical_columns = df.select_dtypes(exclude=["int", "float"]).columns.tolist()
numerical_columns = df.select_dtypes(include=["int", "float"]).columns.tolist()

# Remover 'price' (variável alvo) das colunas numéricas
numerical_columns.remove('price')

# Separar X e y
X = df.drop(columns=["price"])
y = df["price"]

# 2. Aplicar o OneHotEncoder nas colunas categóricas
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
encoded_data = ohe.fit_transform(X[categorical_columns])

# Criar DataFrame com os dados codificados
encoded_df = pd.DataFrame(encoded_data, columns=ohe.get_feature_names_out(categorical_columns), index=df.index)

# Concatenar as colunas codificadas com as numéricas
X = X.drop(columns=categorical_columns)
X = pd.concat([X, encoded_df], axis=1)

# 3. Aplicar o StandardScaler nas colunas numéricas (incluindo as novas features numéricas)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X[numerical_columns])

# Criar um DataFrame com os dados escalonados
scaled_df = pd.DataFrame(scaled_data, columns=numerical_columns, index=df.index)

# Substituir as colunas numéricas originais pelos valores escalonados
X[numerical_columns] = scaled_df

# 4. Dividir os dados em treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# 5. Treinar o modelo
randfor_model = RandomForestRegressor()
randfor_model.fit(X_train, y_train)

# 6. Salvar os artefatos
with open('onehotencoder.pkl', 'wb') as file:
    pickle.dump(ohe, file)

with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

with open('randfor_model.pkl', 'wb') as file:
    pickle.dump(randfor_model, file)

print("Artefatos salvos com sucesso!")

Artefatos salvos com sucesso!
